In [1]:
import toml
import pathlib

from pathlib import Path
import numpy as np
import torch

import pyworld as pw
import pysptk as ps
import librosa
import soundfile as sf
import cv2

from cmvc import *
from cmvc.utils.data.dataset import PairDataset
from cmvc.utils.data.voice import Wave

In [2]:
import IPython.display as display

In [3]:
dict_toml = toml.load(open('/home/jun/Documents/CMVC/cmvc/config.toml'))

image_path = pathlib.Path(dict_toml["path"]["dataset"]["processing"]["image"])
voice_path = pathlib.Path(dict_toml["path"]["dataset"]["processing"]["voice"])

# 第0段階

In [4]:
uttr_path = pathlib.Path(dict_toml["path"]["dataset"]["voice"])/"vcc2018_database_evaluation/vcc2018_evaluation"/"VCC2SF1"

In [5]:
wav_file = list(uttr_path.iterdir())[0]

In [6]:
wav_file

PosixPath('/mnt/f/dataset/DS_10283_3061/vcc2018_database_evaluation/vcc2018_evaluation/VCC2SF1/30001.wav')

In [7]:
display.Audio(wav_file)

# sp確認

In [8]:
x, fs = librosa.load(wav_file, sr=22050, dtype=np.float64)

In [9]:
f0, sp, ap = pw.wav2world(x, fs)

In [10]:
ps.conversion.sp2mc(sp, 35, 0.46)

array([[-9.55514921e+00,  1.39393476e+00,  5.33872900e-01, ...,
         1.12906970e-03,  8.16117716e-03, -1.95938195e-02],
       [-1.01471222e+01,  1.68672821e+00,  1.27689732e+00, ...,
         6.46439102e-03, -3.13413583e-02,  4.77843685e-02],
       [-1.05493573e+01,  1.39222802e+00,  8.11771230e-01, ...,
        -5.49176992e-03, -2.52393487e-02,  4.38811012e-02],
       ...,
       [-1.04423796e+01,  1.50240559e+00,  1.17143702e+00, ...,
        -1.78606898e-02,  3.72490792e-02, -3.61275998e-02],
       [-1.02579035e+01,  1.58715375e+00,  1.08348813e+00, ...,
         2.77605147e-02,  3.17382114e-02, -4.90581251e-02],
       [-1.06180851e+01,  9.27877245e-01,  1.08135181e+00, ...,
         6.84127340e-03, -3.21803355e-02,  2.64640309e-02]])

# 第1段階

In [11]:
x, fs = librosa.load(wav_file, sr=22050, dtype=np.float64)

In [12]:
f0, sp, ap = pw.wav2world(x, fs)

In [13]:
mc = ps.conversion.sp2mc(sp, 35, 0.46)

In [14]:
c_sp = ps.conversion.mc2sp(mc=mc, alpha=0.46, fftlen=1024)

In [15]:
y = pw.synthesize(f0, c_sp, ap, fs)

In [16]:
display.Audio(y, rate = fs)

# 第2段階

In [17]:
c = Wave("30001.wav", "VCC2SF1", "VCC2SF1/30001.wav", Path("/mnt/f/dataset/DS_10283_3061/vcc2018_database_evaluation/vcc2018_evaluation"), ex_mc=True, ex_mfcc=True)

In [20]:
c.ampl.shape

(151140,)

In [18]:
c_sp = ps.conversion.mc2sp(mc=c.mc, alpha=0.46, fftlen=1024)

In [19]:
y = pw.synthesize(c.f0, c_sp, c.ap, c.fs)

In [20]:
display.Audio(y, rate = c.fs)

# 第3段階

In [4]:
dataset = PairDataset(voice_path=voice_path, image_path=image_path, train=False)

In [52]:
[print(i) for i,x in enumerate(dataset.voice_data) if x.wave_name == "30001.wav" and x.person_name == "VCC2SF1"]

58


[None]

In [5]:
c = dataset.voice_data[58]

In [27]:
c.mfcc.shape

(36, 296)

In [55]:
c_sp = ps.conversion.mc2sp(mc=c.mc, alpha=0.46, fftlen=1024)

In [56]:
y = pw.synthesize(c.f0, c_sp, c.ap, c.fs)

In [57]:
display.Audio(y, rate = c.fs)